In [1]:
import base64
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

In [4]:

SYSTEM_PROMPT = """
You are a witty assistant that will use the chat history and the image 
provided by the user to answer its questions.

Use few words on your answers. Go straight to the point. Do not use any
emoticons or emojis. Do not ask the user any questions.

Be friendly and helpful. Show some personality. Do not be too formal.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=SYSTEM_PROMPT),
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "human",
            [
                {"type": "text", "text": "{prompt}"},
                {
                    "type": "image_url",
                    "image_url": "data:image/jpeg;base64,{image_base64}",
                },
            ],
        ),
    ]
)

In [5]:
chain = prompt_template | model | parser
chain = RunnableWithMessageHistory(
    chain,
    lambda _: ChatMessageHistory(),
    input_messages_key="prompt",
    history_messages_key="chat_history",
)

In [6]:
question = """
What do you see on this image?
"""
image = encode_image('ref_data/Starter image.jpeg')

In [7]:
for s in chain.stream({"prompt": question, "image_base64": image},
                      config={"configurable": {"session_id": "unused"}}):
    print(s, end="", flush=True)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


The image shows a diagram of an engine with a focus on the starter motor, labeled as "1" with the word "Стартер," which means "starter" in Russian.